Objective:

By the end of this notebook, you will be able to:


*   Load structured data into PySpark DataFrames from various file formats
*   Use createOrReplaceTempView() to register a DataFrame as a temporary SQL view
*   Write and execute SQL queries within PySpark to: Count total records,  Filter data using logical conditions, Group data and compute aggregates

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
    .appName("Iris Data Comparison") \
    .getOrCreate()

In [ ]:
# Load datasets using Spark
iris_csv_spark = spark.read.csv("/content/drive/MyDrive/Datasets/Iris.csv", header=True, inferSchema=True)
iris_json_spark = spark.read.json("/content/drive/MyDrive/Datasets/Iris.json")
iris_parquet_spark = spark.read.parquet("/content/drive/MyDrive/Datasets/Iris.parquet")

# Show a few rows
print("CSV Sample (Spark):")
iris_csv_spark.show(5)

CSV Sample (Spark):
+------------+-----------+------------+-----------+-------+
|sepal.length|sepal.width|petal.length|petal.width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [ ]:
print("\nSpark CSV Schema:")
iris_csv_spark.printSchema()

print("\nSpark JSON Schema:")
iris_json_spark.printSchema()

print("\nSpark Parquet Schema:")
iris_parquet_spark.printSchema()


Spark CSV Schema:
root
 |-- sepal.length: double (nullable = true)
 |-- sepal.width: double (nullable = true)
 |-- petal.length: double (nullable = true)
 |-- petal.width: double (nullable = true)
 |-- variety: string (nullable = true)


Spark JSON Schema:
root
 |-- petal.length: double (nullable = true)
 |-- petal.width: double (nullable = true)
 |-- sepal.length: double (nullable = true)
 |-- sepal.width: double (nullable = true)
 |-- variety: string (nullable = true)


Spark Parquet Schema:
root
 |-- sepal.length: double (nullable = true)
 |-- sepal.width: double (nullable = true)
 |-- petal.length: double (nullable = true)
 |-- petal.width: double (nullable = true)
 |-- variety: string (nullable = true)



In [ ]:
for col_name in iris_csv_spark.columns:
    print(col_name)

sepal.length
sepal.width
petal.length
petal.width
variety


In [ ]:
iris_csv_spark = iris_csv_spark.withColumnRenamed("sepal.width", "sepal_width")

In [ ]:
iris_csv_spark = iris_csv_spark.withColumnRenamed("sepal.length", "sepal_length")
iris_csv_spark = iris_csv_spark.withColumnRenamed("petal.length", "petal_length")
iris_csv_spark = iris_csv_spark.withColumnRenamed("petal.width", "petal_width")

In [ ]:
# Choose one format for the demo
df = iris_csv_spark
df.show(5)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [ ]:
# Register the DataFrame as a SQL temp view
df.createOrReplaceTempView("iris")

In [ ]:
# SQL query to filter rows
filtered = spark.sql("""
    SELECT * FROM iris
    WHERE petal_length > 1.50
""")
filtered.show(5)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.4|        3.9|         1.7|        0.4| Setosa|
|         4.8|        3.4|         1.6|        0.2| Setosa|
|         5.7|        3.8|         1.7|        0.3| Setosa|
|         5.4|        3.4|         1.7|        0.2| Setosa|
|         5.1|        3.3|         1.7|        0.5| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [ ]:
# SQL group-by with average aggregation
avg_sepal_width = spark.sql("""
    SELECT variety, ROUND(AVG(sepal_width), 2) AS avg_sepal_width
    FROM iris
    GROUP BY variety
    ORDER BY variety
""")
avg_sepal_width.show()

+----------+---------------+
|   variety|avg_sepal_width|
+----------+---------------+
|    Setosa|           3.43|
|Versicolor|           2.77|
| Virginica|           2.97|
+----------+---------------+

